In [1]:
import json
import time

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [4]:
from tqdm import tqdm_notebook

In [5]:
base_url = "https://www.trustpilot.com"

In [6]:
def get_soup(url):
    return BeautifulSoup(requests.get(url).content, 'lxml')

In [7]:
data = {}

soup = get_soup(base_url + '/categories')
for category in soup.findAll('div', {'class' : 'subCategory___BRUDy'}):
    data[category['id']] = {}
    for sub_category in category.findAll('div', {'class' : 'subCategoryItem___3ksKz'}):
        '''Note : .find() will search and return the first occurence but .findAll() will return all occurences'''
        name = sub_category.find('span').text
        link = sub_category.find('a', {'class' : 'internal___1jK0Z typography___lxzyt weight-inherit___229vl navigation___2n5P8'})['href']
        data[category['id']][name] = link
        
''' data will have all sub-catogaries in JSON format 👍'''

' data will have all sub-catogaries in JSON format 👍'

In [8]:
def fetch_company_urls_form_page():
    a_list = driver.find_elements_by_xpath('//a[@class="internal___1jK0Z wrapper___26yB4"]')
    urls=[]
    for a in a_list:
        link = str(a.get_attribute('href'))
        if link.startswith('https://www.trustpilot.com/review/'):
            urls.append(link)
    dedup_urls = list(set(urls))
    return dedup_urls

In [9]:
def go_next_page():
    try:
        button = driver.find_element_by_xpath('//a[@class="paginationLinkNormalize___scOgG paginationLinkNext___1LQ14"]')
        return True, button
    except NoSuchElementException:
        print('No Next Page')
        return False, None

In [10]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument("--disable-extensions")

prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome('./driver/chromedriver', options=options)

timeout = 3

In [11]:
company_urls = {}
wrong_redirects = []

In [33]:
for category in tqdm_notebook(data):
    for sub_category in tqdm_notebook(data[category], leave=False):
        
        ''' 0. Checking if sub-category already present. Useful in case of power faliure and re-running of this cell '''
        
        if ((sub_category in company_urls) and (company_urls[sub_category] != '')):
            print('Already Present : ', sub_category)
            continue
        else:
            company_urls[sub_category] = []
        
        ''' 1. Fetching URLs for each sub-category '''
        
        url = base_url + data[category][sub_category] + "?numberofreviews=0&timeperiod=0&status=all"
        print('URL = ',url)
        
        ''' 2. Printing Info '''
        
        print ("Current Sub-Category : ", category, '->', sub_category)
        driver.get(url)
        current_url = driver.current_url
        
        ''' 3. Checking for bad URL redirects '''
        
        if (url != current_url):
            print('Pre URL : ', url, '\nRedirected URL : ', current_url)
            wrong_redirects.append(sub_category)
            print('Bad URL redirect with : ', sub_category)
            break
            
        ''' 4. Check if current page has URLs for companies, else continue to next sub-category '''            
        try: 
            element_present = EC.presence_of_element_located(
                (By.CLASS_NAME, 'businessUnitCardsContainer___Qhix1'))
            WebDriverWait(driver, timeout).until(element_present)
        except:
            pass
        
        ''' 5. Add all review links in page to list '''   
        
        next_page = True
        count = 1
        
        while next_page:
            print('Current Page URL in Loop : ', driver.current_url)
            extracted_company_urls = fetch_company_urls_form_page()
            print("Scanned Page", count, "of ", sub_category, "URLs found : ", len(extracted_company_urls))
            company_urls[sub_category] += extracted_company_urls
            next_page, button = go_next_page()
            
            if next_page:
                count += 1
                next_url = base_url + data[category][sub_category] + "?numberofreviews=0&timeperiod=0&status=all" + f'&page={count}'
                driver.get(next_url)
                try: 
                    element_present = EC.presence_of_element_located(
                        (By.CLASS_NAME, 'businessUnitCardsContainer___Qhix1'))
                    WebDriverWait(driver, timeout).until(element_present)
                except:
                    print('No next page left')
                    pass

C:\Users\teeji\anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


C:\Users\teeji\anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Already Present :  Animal Health
Already Present :  Animal Parks & Zoo
Already Present :  Cats & Dogs
Already Present :  Horses & Riding
Already Present :  Pet Services
Already Present :  Pet Stores


Already Present :  Cosmetics & Makeup
Already Present :  Hair Care & Styling
Already Present :  Personal Care
Already Present :  Salons & Clinics
Already Present :  Tattoos & Piercings
Already Present :  Wellness & Spa
Already Present :  Yoga & Meditation


Already Present :  Administration & Services
Already Present :  Associations & Centers
Already Present :  HR & Recruiting
Already Present :  IT & Communication
Already Present :  Import & Export
Already Present :  Office Space & Supplies
Already Present :  Print & Graphic Design
Already Present :  Research & Development
Already Present :  Sales & Marketing
Already Present :  Shipping & Logistics
Already Present :  Wholesale


...

Already Present :  Accessories
Already Present :  Clothing & Underwear
Already Present :  Clothing Rental & Repair
Already Present :  Costume & Wedding
Already Present :  Jewelry & Watches
Already Present :  Malls & Marketplaces


Already Present :  Ball Games
Already Present :  Bat-and-ball Games
Already Present :  Bowls & Lawn Sports
Already Present :  Dancing & Gymnastics
Already Present :  Equipment & Associations
Already Present :  Extreme Sports
Already Present :  Fitness & Weight Lifting
Already Present :  Golf & Ultimate
Already Present :  Hockey & Ice Skating
Already Present :  Martial arts & Wrestling
Already Present :  Outdoor & Winter Sports
Already Present :  Shooting & Target Sports
Already Present :  Swimming & Water Sports
Already Present :  Tennis & Racquet Sports


Already Present :  Accommodation & Lodging
Already Present :  Activities & Tours
Already Present :  Airlines & Air Travel
Already Present :  Hotels
Already Present :  Travel Agencies


Already Present :  Energy & Power
Already Present :  Water Utilities


Already Present :  Air & Water Transport
Already Present :  Airports & Parking
Already Present :  Auto Parts & Wheels
Already Present :  Bicycles
Already Present :  Cars & Trucks
Already Present :  Motorcycle & Powersports
Already Present :  Other Vehicles & Trailers
Already Present :  Taxis & Public Transport
Already Present :  Vehical Rental
Already Present :  Vehicle Repair & Fuel



In [34]:
''' Checking list of keys '''
company_urls.keys()

dict_keys(['Animal Health', 'Animal Parks & Zoo', 'Cats & Dogs', 'Horses & Riding', 'Pet Services', 'Pet Stores', 'Cosmetics & Makeup', 'Hair Care & Styling', 'Personal Care', 'Salons & Clinics', 'Tattoos & Piercings', 'Wellness & Spa', 'Yoga & Meditation', 'Administration & Services', 'Associations & Centers', 'HR & Recruiting', 'IT & Communication', 'Import & Export', 'Office Space & Supplies', 'Print & Graphic Design', 'Research & Development', 'Sales & Marketing', 'Shipping & Logistics', 'Wholesale', 'Architects & Engineers', 'Building Materials', 'Chemicals & Plastic', 'Construction Services', 'Contractors & Consultants', 'Factory Equipment', 'Garden & Landscaping', 'Industrial Supplies', 'Manufacturing', 'Production Services', 'Tools & Equipment', 'Colleges & Universities', 'Courses & Classes', 'Education Services', 'Language Learning', 'Music & Theater Classes', 'School & High School', 'Specials Schools', 'Vocational Training', 'Appliances & Electronics', 'Audio & Visual', 'Comp

In [35]:
print(wrong_redirects)

['Gambling', 'Butcher', 'Hotels', 'Bicycles', 'Gaming', 'Travel Agencies']


In [23]:
''' Re-run the above cell [12] to fix breakage due to wrong redirects. We will let go of the reviews for these sub-categories for now 😿 '''

' Re-run the above cell [12] to fix breakage due to wrong redirects. We will let go of the reviews for these sub-categories for now 😿 '

In [ ]:
''' Saving all the scraped data in dataframe '''

consolidated_data = []

for category in data:
    for sub_category in data[category]:
        for url in company_urls[sub_category]:
            consolidated_data.append((category, sub_category, url))

df_consolidated_data = pd.DataFrame(consolidated_data, columns=['category', 'sub_category', 'company_url'])

df_consolidated_data.to_csv('./exports/consolidate_company_urls.csv', index=False)

In [38]:
''' checking our dataframe '''

df_consolidated_data.head()

,category,sub_category,company_url
0,animals_pets,Animal Health,https://www.trustpilot.com/review/www.vetappro...
1,animals_pets,Animal Health,https://www.trustpilot.com/review/gallant.com
2,animals_pets,Animal Health,https://www.trustpilot.com/review/www.mosaicti...
3,animals_pets,Animal Health,https://www.trustpilot.com/review/vetpetni.co.uk
4,animals_pets,Animal Health,https://www.trustpilot.com/review/caninehealth...
